In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

In [ ]:
# read in Fast Food & Obesity csv files - Anne

#connect to Census API - Julia 

#convert Census API zip code into state

#merge tables by state 

#create data frame including state, fast food chain, obesity rate

#data frame population and state



In [ ]:
### Data Cleanup 
# Census data

#figure out census data - class info



In [ ]:
### Data Cleanup 
# Obesity data

# read in csv file - Anne